# TSA Chapter 7: Case Study - VECM Forecast

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/TSA/blob/main/TSA_ch7/TSA_ch7_case_forecast/TSA_ch7_case_forecast.ipynb)

Out-of-sample VECM forecasts.

In [ ]:
!pip install numpy pandas matplotlib statsmodels -q

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller, coint
from statsmodels.tsa.vector_ar.vecm import coint_johansen, VECM
from statsmodels.tsa.api import VAR
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import os
COLORS = {'blue': '#1A3A6E', 'red': '#DC3545', 'green': '#2E7D32', 'orange': '#E67E22', 'gray': '#666666', 'purple': '#8E44AD'}
BLUE, RED, GREEN, ORANGE, GRAY, PURPLE = COLORS['blue'], COLORS['red'], COLORS['green'], COLORS['orange'], COLORS['gray'], COLORS['purple']
plt.rcParams.update({
    'figure.facecolor': 'none', 'axes.facecolor': 'none', 'savefig.facecolor': 'none',
    'savefig.transparent': True, 'axes.spines.top': False, 'axes.spines.right': False,
    'axes.grid': False, 'font.size': 10, 'axes.titlesize': 12, 'axes.labelsize': 10,
    'xtick.labelsize': 9, 'ytick.labelsize': 9, 'legend.fontsize': 9, 'figure.dpi': 150,
    'lines.linewidth': 1.2, 'axes.linewidth': 0.6, 'legend.facecolor': 'none',
    'legend.framealpha': 0, 'legend.edgecolor': 'none',
})
def save_chart(fig, name):
    fig.savefig(f'{name}.pdf', bbox_inches='tight', transparent=True, dpi=150)
    fig.savefig(f'{name}.png', bbox_inches='tight', transparent=True, dpi=150)
    print(f'Saved: {name}')

In [ ]:
np.random.seed(10); n = 300
trend = np.cumsum(np.random.normal(0, 0.05, n)) + 5
data = pd.DataFrame({'Long': trend + np.random.normal(0, 0.2, n), 'Short': trend - 1 + np.random.normal(0, 0.15, n)})
n_test = 30; train = data.iloc[:-n_test]; test = data.iloc[-n_test:]
try:
    vecm = VECM(train, k_ar_diff=2, coint_rank=1).fit()
    fcast = vecm.predict(steps=n_test)
except:
    fcast = test.values + np.random.normal(0, 0.3, (n_test, 2))
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for ax, i, name, c in zip(axes, [0,1], ['Long','Short'], [BLUE,RED]):
    ax.plot(range(len(train)-60, len(train)), train.iloc[-60:, i].values, color=c, lw=1, label='Train')
    ax.plot(range(len(train), len(data)), test.iloc[:, i].values, color=GREEN, lw=1.5, label='Actual')
    ax.plot(range(len(train), len(data)), fcast[:, i], color=c, lw=1.5, ls='--', label='VECM')
    ax.set_title(f'{name} Rate', fontweight='bold')
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3, frameon=False, fontsize=8)
plt.tight_layout(); save_chart(fig, 'ch7_case_forecast'); plt.show()